### Tutorial 05: Loading the checkpoints and Testing Models 

In this tutorial, we'll go over how to load a saved model checkpoint and test it on a dataset. Checkpoints are useful for saving the progress of training and resuming later, or for testing a trained model on unseen data. We’ll use PyTorch to demonstrate how to save, load, and test a model.

1. **Saving the entire model**
2. **Saving only the model parameters (state_dict)**

In [3]:
import torch
import torch.nn as nn


def load_checkpoint(filepath):
    checkpoint = torch.load(filepath, weights_only=True)  
    input_size = checkpoint['input_size']
    output_size = checkpoint['output_size']
    hidden_layers = checkpoint['hidden_layers']

    # Recreate the model with nn.Sequential based on the saved architecture
    model = nn.Sequential(
        nn.Linear(input_size, hidden_layers[0]),  
        nn.ReLU(),
        nn.Linear(hidden_layers[0], output_size)  
    )
    
    # Load the saved model state_dict (weights)
    model.load_state_dict(checkpoint['state_dict'])
    
    # Set the model to evaluation mode (optional, but recommended for inference)
    model.eval()

    return model


model_path = "./models/test_model.pth"
loaded_model = load_checkpoint(model_path)


In [4]:
import pandas as pd
import numpy as np

unseen_data = pd.DataFrame({
    "x1": [2.0, 3.5, 1.8, 2.5, 3.0],
    "x2": [3000, 4000, 2800, 3500, 3700],
    "x3": [4, 6, 4, 6, 6]   
})

# Normalize the unseen data (same way as you normalized the training data)
X_unseen = unseen_data.values
X_unseen_normalized = (X_unseen - X_unseen.mean(axis=0)) / X_unseen.std(axis=0)

# Convert to tensor
X_unseen_tensor = torch.tensor(X_unseen_normalized, dtype=torch.float32)

with torch.no_grad():  # Disable gradient computation for inference
    predictions = loaded_model(X_unseen_tensor)

# Print the predictions
print("Predictions on unseen data:", predictions.detach().flatten())

Predictions on unseen data: tensor([30.4847, 19.7615, 34.5468, 24.9005, 22.3329])
